# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [5]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [6]:

#Baseline model
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Metal device set to: Apple M1 Pro
Epoch 1/150


2023-06-21 13:07:50.124979: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-21 13:07:50.125259: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-06-21 13:07:50.261231: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


13/33 [==========>...................] - ETA: 0s - loss: nan - mse: nan                          

2023-06-21 13:07:50.355492: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 9ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
26/33 [======================>.......] - ETA: 0s - loss: nan - mse: nan

2023-06-21 13:07:50.687109: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 10/150
33/33 [==============================] - 0s 5ms

33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 78/150
33/33 [==============================] -

33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 146/150
33/33 [=========================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [7]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [9]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 
Again, you may get some strange behavior.

In [10]:
# Train the model
normalized_input_model.fit(X_train,  
                           y_train, 
                           batch_size=32, 
                           epochs=150, 
                           validation_data=(X_val, y_val))# Train the model

Epoch 1/150
25/33 [=====================>........] - ETA: 0s - loss: nan - mse: nan                         

2023-06-21 13:09:40.148838: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 8ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
26/33 [======================>.......] - ETA: 0s - loss: nan - mse: nan

2023-06-21 13:09:40.419687: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 10/150
33/33 [==============================] - 0s 5ms

33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 78/150
33/33 [==============================] -

33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 146/150
33/33 [=========================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [11]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [12]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))

Epoch 1/150
26/33 [======================>.......] - ETA: 0s - loss: 0.5130 - mse: 0.5130

2023-06-21 13:10:43.490866: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 8ms/step - loss: 0.4988 - mse: 0.4988 - val_loss: 0.1796 - val_mse: 0.1796
Epoch 2/150
27/33 [=======================>......] - ETA: 0s - loss: 0.2884 - mse: 0.2884

2023-06-21 13:10:43.755700: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 5ms/step - loss: 0.2591 - mse: 0.2591 - val_loss: 0.1515 - val_mse: 0.1515
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1930 - mse: 0.1930 - val_loss: 0.1373 - val_mse: 0.1373
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1704 - mse: 0.1704 - val_loss: 0.1302 - val_mse: 0.1302
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1449 - mse: 0.1449 - val_loss: 0.1313 - val_mse: 0.1313
Epoch 6/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1269 - mse: 0.1269 - val_loss: 0.1257 - val_mse: 0.1257
Epoch 7/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1170 - mse: 0.1170 - val_loss: 0.1230 - val_mse: 0.1230
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1069 - mse: 0.1069 - val_loss: 0.1249 - val_mse: 0.1249
Epoch 9/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1002 - mse: 0.1002 -

33/33 [==============================] - 0s 5ms/step - loss: 0.0195 - mse: 0.0195 - val_loss: 0.1106 - val_mse: 0.1106
Epoch 65/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0190 - mse: 0.0190 - val_loss: 0.1120 - val_mse: 0.1120
Epoch 66/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0192 - mse: 0.0192 - val_loss: 0.1092 - val_mse: 0.1092
Epoch 67/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0183 - mse: 0.0183 - val_loss: 0.1104 - val_mse: 0.1104
Epoch 68/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0183 - mse: 0.0183 - val_loss: 0.1093 - val_mse: 0.1093
Epoch 69/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0180 - mse: 0.0180 - val_loss: 0.1095 - val_mse: 0.1095
Epoch 70/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0178 - mse: 0.0178 - val_loss: 0.1089 - val_mse: 0.1089
Epoch 71/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0172 - mse: 0

Epoch 126/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0086 - mse: 0.0086 - val_loss: 0.1079 - val_mse: 0.1079
Epoch 127/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0085 - mse: 0.0085 - val_loss: 0.1083 - val_mse: 0.1083
Epoch 128/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0085 - mse: 0.0085 - val_loss: 0.1087 - val_mse: 0.1087
Epoch 129/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0084 - mse: 0.0084 - val_loss: 0.1083 - val_mse: 0.1083
Epoch 130/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0081 - mse: 0.0081 - val_loss: 0.1088 - val_mse: 0.1088
Epoch 131/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0081 - mse: 0.0081 - val_loss: 0.1090 - val_mse: 0.1090
Epoch 132/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0081 - mse: 0.0081 - val_loss: 0.1089 - val_mse: 0.1089
Epoch 133/150
33/33 [==============================] - 0s 5ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [13]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 3ms/step - loss: 0.0065 - mse: 0.0065


[0.0065474119037389755, 0.0065474119037389755]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [14]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 4ms/step - loss: 0.1093 - mse: 0.1093


[0.10926736146211624, 0.10926736146211624]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [15]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

9/9 [==============================] - 0s 3ms/step


2023-06-21 13:11:18.687683: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


25976.266357408622

Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'` 
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [16]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, kernel_initializer='he_normal', activation='relu', input_shape=(n_features)))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150
26/33 [======================>.......] - ETA: 0s - loss: 0.4861 - mse: 0.4861

2023-06-21 13:11:30.913414: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 8ms/step - loss: 0.4854 - mse: 0.4854 - val_loss: 0.2083 - val_mse: 0.2083
Epoch 2/150
26/33 [======================>.......] - ETA: 0s - loss: 0.2497 - mse: 0.2497

2023-06-21 13:11:31.180608: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 5ms/step - loss: 0.2473 - mse: 0.2473 - val_loss: 0.1742 - val_mse: 0.1742
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1956 - mse: 0.1956 - val_loss: 0.1588 - val_mse: 0.1588
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1749 - mse: 0.1749 - val_loss: 0.1610 - val_mse: 0.1610
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1522 - mse: 0.1522 - val_loss: 0.1469 - val_mse: 0.1469
Epoch 6/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1361 - mse: 0.1361 - val_loss: 0.1466 - val_mse: 0.1466
Epoch 7/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1297 - mse: 0.1297 - val_loss: 0.1411 - val_mse: 0.1411
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1166 - mse: 0.1166 - val_loss: 0.1433 - val_mse: 0.1433
Epoch 9/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1045 - mse: 0.1045 -

33/33 [==============================] - 0s 5ms/step - loss: 0.0243 - mse: 0.0243 - val_loss: 0.1470 - val_mse: 0.1470
Epoch 65/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0245 - mse: 0.0245 - val_loss: 0.1447 - val_mse: 0.1447
Epoch 66/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0236 - mse: 0.0236 - val_loss: 0.1477 - val_mse: 0.1477
Epoch 67/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0237 - mse: 0.0237 - val_loss: 0.1452 - val_mse: 0.1452
Epoch 68/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0231 - mse: 0.0231 - val_loss: 0.1460 - val_mse: 0.1460
Epoch 69/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0227 - mse: 0.0227 - val_loss: 0.1459 - val_mse: 0.1459
Epoch 70/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0222 - mse: 0.0222 - val_loss: 0.1454 - val_mse: 0.1454
Epoch 71/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0219 - mse: 0

Epoch 126/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1474 - val_mse: 0.1474
Epoch 127/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1490 - val_mse: 0.1490
Epoch 128/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.1493 - val_mse: 0.1493
Epoch 129/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.1495 - val_mse: 0.1495
Epoch 130/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1489 - val_mse: 0.1489
Epoch 131/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1495 - val_mse: 0.1495
Epoch 132/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1496 - val_mse: 0.1496
Epoch 133/150
33/33 [==============================] - 0s 5ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [17]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 3ms/step - loss: 0.0085 - mse: 0.0085


[0.008450852707028389, 0.008450852707028389]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [18]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 4ms/step - loss: 0.1501 - mse: 0.1501


[0.15014097094535828, 0.15014097094535828]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [19]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=(n_features)))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
26/33 [======================>.......] - ETA: 0s - loss: 0.4726 - mse: 0.4726

2023-06-21 13:12:06.590839: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 9ms/step - loss: 0.4258 - mse: 0.4258 - val_loss: 0.1528 - val_mse: 0.1528
Epoch 2/150
27/33 [=======================>......] - ETA: 0s - loss: 0.1524 - mse: 0.1524

2023-06-21 13:12:06.873200: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 5ms/step - loss: 0.2354 - mse: 0.2354 - val_loss: 0.2437 - val_mse: 0.2437
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1988 - mse: 0.1988 - val_loss: 0.1164 - val_mse: 0.1164
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1791 - mse: 0.1791 - val_loss: 0.1194 - val_mse: 0.1194
Epoch 5/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1592 - mse: 0.1592 - val_loss: 0.1192 - val_mse: 0.1192
Epoch 6/150
33/33 [==============================] - 0s 7ms/step - loss: 0.1458 - mse: 0.1458 - val_loss: 0.1099 - val_mse: 0.1099
Epoch 7/150
33/33 [==============================] - 0s 7ms/step - loss: 0.1304 - mse: 0.1304 - val_loss: 0.1108 - val_mse: 0.1108
Epoch 8/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1240 - mse: 0.1240 - val_loss: 0.1112 - val_mse: 0.1112
Epoch 9/150
33/33 [==============================] - 0s 7ms/step - loss: 0.1127 - mse: 0.1127 -

33/33 [==============================] - 0s 5ms/step - loss: 0.0196 - mse: 0.0196 - val_loss: 0.1428 - val_mse: 0.1428
Epoch 65/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0194 - mse: 0.0194 - val_loss: 0.1465 - val_mse: 0.1465
Epoch 66/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0189 - mse: 0.0189 - val_loss: 0.1448 - val_mse: 0.1448
Epoch 67/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0188 - mse: 0.0188 - val_loss: 0.1467 - val_mse: 0.1467
Epoch 68/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0182 - mse: 0.0182 - val_loss: 0.1459 - val_mse: 0.1459
Epoch 69/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0179 - mse: 0.0179 - val_loss: 0.1424 - val_mse: 0.1424
Epoch 70/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0176 - mse: 0.0176 - val_loss: 0.1451 - val_mse: 0.1451
Epoch 71/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0174 - mse: 0

Epoch 126/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0083 - mse: 0.0083 - val_loss: 0.1546 - val_mse: 0.1546
Epoch 127/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0079 - mse: 0.0079 - val_loss: 0.1564 - val_mse: 0.1564
Epoch 128/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0080 - mse: 0.0080 - val_loss: 0.1552 - val_mse: 0.1552
Epoch 129/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0076 - mse: 0.0076 - val_loss: 0.1543 - val_mse: 0.1543
Epoch 130/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0076 - mse: 0.0076 - val_loss: 0.1554 - val_mse: 0.1554
Epoch 131/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0076 - mse: 0.0076 - val_loss: 0.1552 - val_mse: 0.1552
Epoch 132/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0075 - mse: 0.0075 - val_loss: 0.1534 - val_mse: 0.1534
Epoch 133/150
33/33 [==============================] - 0s 5ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [20]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 4ms/step - loss: 0.0056 - mse: 0.0056


[0.0055966624058783054, 0.0055966624058783054]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 4ms/step - loss: 0.1567 - mse: 0.1567


[0.15667259693145752, 0.15667259693145752]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [22]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop', 
                      loss='mse', 
                      metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
 1/33 [..............................] - ETA: 10s - loss: 2.0703 - mse: 2.0703

2023-06-21 13:12:57.082221: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 1s 11ms/step - loss: 0.4452 - mse: 0.4452 - val_loss: 0.1292 - val_mse: 0.1292
Epoch 2/150
19/33 [================>.............] - ETA: 0s - loss: 0.1871 - mse: 0.1871

2023-06-21 13:12:57.523036: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 7ms/step - loss: 0.1765 - mse: 0.1765 - val_loss: 0.1326 - val_mse: 0.1326
Epoch 3/150
33/33 [==============================] - 0s 7ms/step - loss: 0.1240 - mse: 0.1240 - val_loss: 0.1050 - val_mse: 0.1050
Epoch 4/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0965 - mse: 0.0965 - val_loss: 0.1187 - val_mse: 0.1187
Epoch 5/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0812 - mse: 0.0812 - val_loss: 0.1016 - val_mse: 0.1016
Epoch 6/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0610 - mse: 0.0610 - val_loss: 0.1149 - val_mse: 0.1149
Epoch 7/150
33/33 [==============================] - 0s 9ms/step - loss: 0.0470 - mse: 0.0470 - val_loss: 0.1353 - val_mse: 0.1353
Epoch 8/150
33/33 [==============================] - 0s 9ms/step - loss: 0.0504 - mse: 0.0504 - val_loss: 0.1362 - val_mse: 0.1362
Epoch 9/150
33/33 [==============================] - 0s 8ms/step - loss: 0.0376 - mse: 0.0376 -

33/33 [==============================] - 0s 6ms/step - loss: 0.0096 - mse: 0.0096 - val_loss: 0.0998 - val_mse: 0.0998
Epoch 65/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0065 - mse: 0.0065 - val_loss: 0.1053 - val_mse: 0.1053
Epoch 66/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0066 - mse: 0.0066 - val_loss: 0.1024 - val_mse: 0.1024
Epoch 67/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0083 - mse: 0.0083 - val_loss: 0.1046 - val_mse: 0.1046
Epoch 68/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0061 - mse: 0.0061 - val_loss: 0.0987 - val_mse: 0.0987
Epoch 69/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0069 - mse: 0.0069 - val_loss: 0.1138 - val_mse: 0.1138
Epoch 70/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0077 - mse: 0.0077 - val_loss: 0.0953 - val_mse: 0.0953
Epoch 71/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0069 - mse: 0

Epoch 126/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0049 - mse: 0.0049 - val_loss: 0.1045 - val_mse: 0.1045
Epoch 127/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0043 - mse: 0.0043 - val_loss: 0.0959 - val_mse: 0.0959
Epoch 128/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0047 - mse: 0.0047 - val_loss: 0.1062 - val_mse: 0.1062
Epoch 129/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0040 - mse: 0.0040 - val_loss: 0.1031 - val_mse: 0.1031
Epoch 130/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0044 - mse: 0.0044 - val_loss: 0.1118 - val_mse: 0.1118
Epoch 131/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0051 - mse: 0.0051 - val_loss: 0.1021 - val_mse: 0.1021
Epoch 132/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0036 - mse: 0.0036 - val_loss: 0.0994 - val_mse: 0.0994
Epoch 133/150
33/33 [==============================] - 0s 6ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [23]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 4ms/step - loss: 0.0026 - mse: 0.0026


[0.0025646057911217213, 0.0025646057911217213]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [24]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 4ms/step - loss: 0.1061 - mse: 0.1061


[0.10605713725090027, 0.10605713725090027]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [25]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam', 
                   loss='mse', 
                   metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
10/33 [========>.....................] - ETA: 0s - loss: 0.7477 - mse: 0.7477

2023-06-21 13:13:39.327663: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 9ms/step - loss: 0.4966 - mse: 0.4966 - val_loss: 0.1564 - val_mse: 0.1564
Epoch 2/150
22/33 [===================>..........] - ETA: 0s - loss: 0.1733 - mse: 0.1733

2023-06-21 13:13:39.653560: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 6ms/step - loss: 0.1752 - mse: 0.1752 - val_loss: 0.1467 - val_mse: 0.1467
Epoch 3/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1243 - mse: 0.1243 - val_loss: 0.1213 - val_mse: 0.1213
Epoch 4/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0927 - mse: 0.0927 - val_loss: 0.1113 - val_mse: 0.1113
Epoch 5/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0688 - mse: 0.0688 - val_loss: 0.1080 - val_mse: 0.1080
Epoch 6/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0530 - mse: 0.0530 - val_loss: 0.1093 - val_mse: 0.1093
Epoch 7/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0430 - mse: 0.0430 - val_loss: 0.1127 - val_mse: 0.1127
Epoch 8/150
33/33 [==============================] - 0s 8ms/step - loss: 0.0340 - mse: 0.0340 - val_loss: 0.1115 - val_mse: 0.1115
Epoch 9/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0291 - mse: 0.0291 -

Epoch 64/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0040 - mse: 0.0040 - val_loss: 0.1147 - val_mse: 0.1147
Epoch 65/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0043 - mse: 0.0043 - val_loss: 0.1024 - val_mse: 0.1024
Epoch 66/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0068 - mse: 0.0068 - val_loss: 0.1115 - val_mse: 0.1115
Epoch 67/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0078 - mse: 0.0078 - val_loss: 0.1077 - val_mse: 0.1077
Epoch 68/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0058 - mse: 0.0058 - val_loss: 0.1127 - val_mse: 0.1127
Epoch 69/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0045 - mse: 0.0045 - val_loss: 0.1092 - val_mse: 0.1092
Epoch 70/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0040 - mse: 0.0040 - val_loss: 0.1069 - val_mse: 0.1069
Epoch 71/150
33/33 [==============================] - 0s 6ms/step - loss: 0.

Epoch 125/150
33/33 [==============================] - 0s 6ms/step - loss: 8.5462e-05 - mse: 8.5462e-05 - val_loss: 0.1041 - val_mse: 0.1041
Epoch 126/150
33/33 [==============================] - 0s 7ms/step - loss: 7.4034e-05 - mse: 7.4034e-05 - val_loss: 0.1037 - val_mse: 0.1037
Epoch 127/150
33/33 [==============================] - 0s 6ms/step - loss: 8.9102e-05 - mse: 8.9102e-05 - val_loss: 0.1039 - val_mse: 0.1039
Epoch 128/150
33/33 [==============================] - 0s 6ms/step - loss: 1.4844e-04 - mse: 1.4844e-04 - val_loss: 0.1035 - val_mse: 0.1035
Epoch 129/150
33/33 [==============================] - 0s 6ms/step - loss: 3.3740e-04 - mse: 3.3740e-04 - val_loss: 0.1047 - val_mse: 0.1047
Epoch 130/150
33/33 [==============================] - 0s 6ms/step - loss: 9.0752e-04 - mse: 9.0752e-04 - val_loss: 0.1070 - val_mse: 0.1070
Epoch 131/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.1034 - val_mse: 0.1034
Epoch 132/150
33/33 [

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [26]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 4ms/step - loss: 0.0012 - mse: 0.0012


[0.0011882828548550606, 0.0011882828548550606]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [27]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 4ms/step - loss: 0.1016 - mse: 0.1016


[0.10155995190143585, 0.10155995190143585]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [28]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 9ms/step - loss: 0.1717 - mse: 0.1717


[0.17174449563026428, 0.17174449563026428]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [29]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

5/5 [==============================] - 0s 5ms/step


2023-06-21 13:14:33.979698: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


32566.636953303096

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.